In [1]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from pprint import pprint
import json
import time
import requests
from requests.auth import HTTPBasicAuth

# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ["GISBASE"] = gisbase
os.environ["ACTINIA_USER"] = 'actinia-gdi'
os.environ["ACTINIA_PASSWORD"] = 'actinia-gdi'
os.environ["AUTH"] = 'actinia-gdi:actinia-gdi'
os.environ["ACTINIA_URL"] = 'http://localhost:8088'

ACTINIA_VERSION = 'v3'
ACTINIA_BASEURL = 'http://localhost:8088'
ACTINIA_URL = ACTINIA_BASEURL + "/api/" + ACTINIA_VERSION
ACTINIA_AUTH = HTTPBasicAuth("actinia-gdi", "actinia-gdi")
sys.path.append(os.path.join(gisbase, "etc", "python"))

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
## Set your grass data location
gj.init("../actinia-core-data/grassdb", "nc_spm_08", "PERMANENT")

# Actinia STAC Managment

Acinita is able to consume public and private Spatial Temporal Asset Catalogs (STAC)

## View STAC instances

Two Actinia routes seem to accomplish this task

### /stac

In [2]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/stac" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   136  100   136    0     0    473      0 --:--:-- --:--:-- --:--:--   472
{
  "defaultStac": {
    "path": "stac.defaultStac.rastercube.<stac_collection_id>"
  },
  "naip": {
    "path": "stac.naip.rastercube.<stac_collection_id>"
  }
}


### /stac/instances

In [3]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/stac/instances" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   136  100   136    0     0    483      0 --:--:-- --:--:-- --:--:--   482
{
  "defaultStac": {
    "path": "stac.defaultStac.rastercube.<stac_collection_id>"
  },
  "naip": {
    "path": "stac.naip.rastercube.<stac_collection_id>"
  }
}


## View STAC Catalogs

### /stac/catalogs/catalog.json

In [ ]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/stac/catalogs/catalog.json" | jq

## View STAC collections

STAC collection spec [https://github.com/radiantearth/stac-spec/blob/master/collection-spec/collection-spec.md](https://github.com/radiantearth/stac-spec/blob/master/collection-spec/collection-spec.md)

### /stac/collections

In [ ]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/stac/collections" | jq

## View STAC collection

### /stac/collections/{stac_collection_id}

In [ ]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/stac/collections/stac.defaultStac.rastercube.landsat-8-l1-c1" | jq

# Demo

We will now add the Planet SkySat STAC from [https://stacindex.org/catalogs/planet-labs-stac-catalog#/](https://stacindex.org/catalogs/planet-labs-stac-catalog#/)

## 1) Add new STAC instance

In [15]:
!curl -u 'actinia-gdi:actinia-gdi' -X POST "http://localhost:8088/api/v3/stac/instances/" -H "Content-Type: application/json" \
   -d '{"stac_instance_id": "planet"}' | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   103  100    73  100    30    231     95 --:--:-- --:--:-- --:--:--   326
{
  "StacInstance": {},
  "message": "The Instance has been added successfully"
}


Now lets view the new instance by requesting it with the **/stac/instances/{stac_instance_id}** GET request.

In [13]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/stac/instances/usgs" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     3  100     3    0     0     10      0 --:--:-- --:--:-- --:--:--    10
{}


The request should return an empty response object because we have not added any collections to the data yet.

## 2) Add a new collection to an instance

To add a new collect to the opentopography instace we need to make a POST request to the **/stac/collections** routes specifying the `stac_instance_id` and setting the `stac_url`.

In our case the `stac_instance_id = opentopography` and the `stac_url = https://www.planet.com/data/stac/open-skysat-data/collection.json`

The request body will look like 

```json
{
    "stac_instance_id": "opentopography", 
    "stac_url": "https://www.planet.com/data/stac/open-skysat-data/collection.json"}
}
```

In [16]:
!curl -u 'actinia-gdi:actinia-gdi' -X POST "http://localhost:8088/api/v3/stac/collections" -H "Content-Type: application/json" \
   -d '{"stac_instance_id": "planet", "stac_url": "https://www.planet.com/data/stac/open-skysat-data/collection.json"}' | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388  100   277  100   111    160     64  0:00:01  0:00:01 --:--:--   225
{
  "StacCollection": {
    "stac.planet.rastercube.planet-stac-skysat": {
      "href": "api/v3/stac/collections/stac.planet.rastercube.planet-stac-skysat",
      "root": "https://www.planet.com/data/stac/open-skysat-data/collection.json"
    }
  },
  "message": "The STAC Collection has been added successfully"
}


## 3) View the new collection

Now lets view the new collection using the GET **/stac/collections/{stac_collection_id}** by setting the route `stac_collection_id` route parameter to `stac.planet.rastercube.planet-stac-skysat`.

In [18]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/stac/collections/stac.planet.rastercube.planet-stac-skysat" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2485  100  2485    0     0   8875      0 --:--:-- --:--:-- --:--:--  8875
{
  "description": "A Catalog of SkySat Data Collected for Winners of the Cloud Native Geospatial Awards. All data CC-BY 4.0",
  "extent": {
    "spatial": {
      "bbox": [
        [
          -4.185538368682979,
          0.2693073386275247,
          172.95515019320533,
          21.89915173899179
        ]
      ]
    },
    "temporal": {
      "interval": [
        [
          "2020-12-11T10:30:40.625000Z",
          "2020-12-13T08:34:52.312000Z"
        ]
      ]
    }
  },
  "id": "stac.planet.rastercube.planet-stac-skysat",
  "license": "CC-BY-4.0",
  "links": [
    {
      "href": "https://www.planet.com/data/stac/catalog.json",
      "rel": "root",
      "type": "application/json"
    },
    {
      "href": "https://www.planet.com/data/stac/op